In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

In [2]:
types= {
    "event": "category", 
    "model": "category", 
    "condition": "category", 
    "color": "category", 
    "storage": "category"
}
df_events = pd.read_csv('./fiuba-trocafone-tp1-final-set/events.csv', dtype=types, low_memory=False)

In [3]:
df_events = df_events[["person",'timestamp','event','campaign_source','search_engine']]

In [4]:
df_events['is_new_session'] = np.where((df_events["event"] == "visited site"),True,False)

In [5]:
checkout_events = df_events.sort_values(by=['person','timestamp','is_new_session'], ascending=[True,True,False])\
        .loc[(df_events["event"] == "checkout") | (df_events["event"] == "ad campaign hit") \
             | (df_events["event"] == "visited site")]

In [6]:
session_id = (checkout_events["event"] == "visited site").cumsum()

checkout_events['session_id'] = session_id

In [7]:
checkouts_per_session = checkout_events[['event','session_id']].loc[(checkout_events["event"] == "checkout")]\
                    .groupby(['session_id'])\
                    .agg({'event' : 'count'})\
                    .rename(columns={'session_id' : 'session_id' , 'event' : 'checkouts_qty'})
checkouts_per_session

,checkouts_qty
session_id,
1,1
2,1
4,1
5,2
6,2
7,1
9,1
10,1
13,1


In [8]:
campaign_source_per_session = checkout_events[['event','session_id','campaign_source']]\
            .loc[(checkout_events["event"] == "ad campaign hit")].groupby(['session_id'])['campaign_source'].first()
campaign_source_per_session

session_id
1             criteo
2             criteo
3             google
4             google
7             google
9             google
10            criteo
12       FacebookAds
13             zanox
14            google
23            google
25            criteo
26            criteo
27            criteo
28            google
29            google
30          rtbhouse
32            google
34            google
35            google
36            google
37            criteo
38            google
39            google
41            google
43            google
44            google
45            google
46            google
47            google
            ...     
87337         google
87338         google
87339         google
87340         google
87341         google
87342         google
87349         google
87350         google
87351          zanox
87352         google
87353         google
87354         google
87355         google
87356         google
87357         google
87358         google
87

In [9]:
checkouts_and_campaign_source_per_session = pd.concat([checkout_events['session_id'],checkouts_per_session,campaign_source_per_session], axis=1)

In [10]:
checkouts_qty_per_campaign = checkouts_and_campaign_source_per_session\
                        .fillna(0)\
                        .groupby(['campaign_source','checkouts_qty'])\
                        .size()
checkouts_qty_per_campaign

campaign_source   checkouts_qty
0                 0.0              191824
                  1.0               10083
                  2.0                 541
                  3.0                 112
                  4.0                  37
                  5.0                  17
                  6.0                   6
                  7.0                   7
                  8.0                   2
                  9.0                   1
                  12.0                  1
                  14.0                  2
Facebook          0.0                  27
                  1.0                  20
                  2.0                   1
FacebookAds       0.0                 233
                  1.0                 257
                  2.0                  19
                  3.0                   2
                  4.0                   1
                  5.0                   1
FacebookSocial    0.0                   5
                  1.0                   2
MA